# Курс Специализация Data Science Дипломный проект. Бриф учебного кейса 
## «Модель прогнозирования стоимости жилья для агентства недвижимости»

В данном ноутбуке мы сделаем следующее:
* Обучим модель при помощи алгоритма Random Forest т.к. он работает быстро и достаточно хорошо
* Сохраним модель и названия колонок в отдельных файлах
* Напишем функцию для предсказания стоимости квартиры по входным параметрам
* Сохраним наиболее привлекательные варианты в json-файл

In [8]:
import random
import numpy as np 
import pandas as pd 
import sys

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm

In [5]:
# зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы
RANDOM_SEED = 42

TEST_SIZE = 0.2

In [6]:
df = pd.read_csv('data/data_model_ecod.csv')
df.head()

,baths,sqft,target,pool_encoded,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min,...,Type_other,Type_ranch,Type_single_family_home,Type_townhouse,state_label,year_ord,city_te,city_cbe,zip_te,zip_cbe
0,0.103448,0.016379,0.556742,False,True,False,False,True,0.62,0.110778,...,0.0,0.0,1.0,0.0,18,201,0.471381,0.436554,0.471381,0.436554
1,0.068966,0.010994,0.412550,False,False,False,False,False,0.50,0.060180,...,0.0,0.0,1.0,0.0,32,201,0.434601,0.436554,0.415715,0.436554
2,0.034483,0.020266,0.325634,False,True,True,False,False,0.48,0.197605,...,0.0,0.0,1.0,0.0,7,152,0.279350,0.436554,0.279350,0.436554
3,0.068966,0.010898,0.415214,False,True,True,True,False,0.40,0.047904,...,0.0,0.0,1.0,0.0,28,201,0.420480,0.436554,0.454036,0.436554
4,0.034483,0.007339,0.891856,False,False,False,True,False,0.38,0.038922,...,0.0,0.0,0.0,0.0,21,147,0.820348,0.436554,0.768997,0.436554


# Обучение модели

In [7]:
y = df.target.values
X = df.drop(['target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, shuffle=True, random_state=RANDOM_SEED)

## Model 3: RandomForestRegressor

In [9]:
# Создаем экземпляр модели RandomForestRegressor
rf_regressor = RandomForestRegressor(random_state=RANDOM_SEED)

# Обучаем модель на обучающих данных
rf_regressor.fit(X_train, y_train)

# Предсказания на обучающих и тестовых данных
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)

# Вычисляем метрики
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.00
Test MSE: 0.01
Train MAE: 0.02
Test MAE: 0.06
Train R2: 0.98
Test R2: 0.83
